In [70]:
#importing libraries
import requests
from bs4 import BeautifulSoup
import json

In [71]:
def get_IMDb_chart_page(category):
    return requests.get(f"https://www.imdb.com/chart/{category}")

In [72]:
def parse_html(response):
    return BeautifulSoup(response.text, 'html.parser')

In [73]:
def Merge(dict1, dict2):
    res = {**dict1, **dict2}
    return res

In [74]:
# Make a request to the IMDb Top 250 movies page
response = get_IMDb_chart_page('top/')

# Parse the HTML content of the page
soup = parse_html(response)

# Find all the movie names and movie links in the page
movie_elements = soup.find_all('td', class_='titleColumn')

# Initialize an empty list to store the review data
review_data = []
links = []

# Loop through the title_columns and extract the film names and links
for movie in movie_elements:
    # Find the film name
    movie_name = movie.find_all('a')[-1].text

    # Find the link to the review
    review_link = 'https://imdb.com' + \
        movie.find('a')['href'] + \
            'reviews/'
            
    rating = soup.find(class_= "ratingColumn imdbRating").text
        
    rating = rating.strip('\n')
    rating = float(rating)
    reviewdict1 = { 'rating': rating,}
    
        # Print the film name and review link
    # print(f'Here is the review page for "{movie_name}": \n {review_link}')
    links.append(review_link)
    
for review in links:
    # Make a request to the review webpage 
    review_page_response = requests.get(f'{review}')
    
    # review_page_response = requests.get(f'https://www.imdb.com{review_link}')
    
    #parse the HTML content
    review_soup = parse_html(review_page_response)
    # Find all the review elements
    review_elements = review_soup.find_all(class_='review-container')

    # Loop through the review elements and extract the review data
    for element in review_elements:
        # Find the title of the review
        review_title = element.find(class_='title').text

        # Find the text of the review
        review_text = element.find(class_='text').text

        # # Find the rating of the review
        # rating = element.find(class_='rating-other-user-rating')[0].get_text().strip()

        
        #Find the date of the review
        review_date = element.find(class_= 'review-date').text

        # Find the user who wrote the review
        review_user = element.find(class_='display-name-link').text

        # Store the review data in a dictionary
        reviewdict = {
            'title': review_title,
            'text': review_text,
            # 'rating': rating,
            'date': review_date,
            'user': review_user
        }
        
        dict3 = Merge(reviewdict, reviewdict1)
        

        # Add the review to the review_data list
        review_data.append(dict3)

In [ ]:
# Save the review data to a JSON file
with open('review_data.json', 'w') as f:
    json.dump(review_data, f)

In [ ]:
# import requests
# import json
# from bs4 import BeautifulSoup

# # Set the base URL for the review webpage
# review_base_url = 'https://www.imdb.com/title/tt0111161/reviews'

# # Set the URL for the "Load more" button
# load_more_url = 'https://www.imdb.com/title/tt0111161/reviews/_ajax'

# # Initialize an empty list to store the review data
# review_data = []

# # Set a flag to indicate whether there are more reviews to load
# more_reviews = True

# while more_reviews:
#     # Make a request to the review webpage
#     response = requests.get(review_base_url)

#     # Parse the HTML content
#     soup = BeautifulSoup(response.text, 'html.parser')

#     # Find all the review elements
#     review_elements = soup.find_all(class_='review-container')

#     # Loop through the review elements and extract the review data
#     for element in review_elements:
#         # Find the title of the review
#         review_title = element.find(class_='title').text

#         # Find the text of the review
#         review_text = element.find(class_='text').text

#         # Find the rating of the review
#         review_rating = element.find(class_='rating').text

#         # Find the user who wrote the review
#         review_user = element.find(class_='display-name-link').text

#         # Store the review data in a dictionary
#         review = {
#             'title': review_title,
#             'text': review_text,
#             'rating': review_rating,
#             'user': review_user
#         }

#         # Add the review to the review_data list
#         review_data.append(review)

#     # Check if there are more reviews to load
#     more_reviews_button = soup.find(class_='ipl-load-more__button')
#     if more_reviews_button:
#         # There are more reviews to load, simulate the "Load more" button request
#         review_count = len(review_data)
#         payload = {'paginationKey': more_reviews_button['data-key']}
#         headers = {'referer': review_base_url}
#         response = requests.post(load_more_url, data=payload, headers=headers)

#         # Update the review webpage URL with the next page of reviews
#         review_base_url = f'{review_base_url}?start={review_count}'
#     else:
#         # There are no more reviews to load, set the flag to False
#         more_reviews = False

# # Save the review data to a JSON file
# with open('review_data.json', 'w') as f:
#     json.dump(review_data,